# Ý tưởng chính
- Tránh việc liên tục tải dữ liệu độc lập mỗi lần chạy jobs, ta sẽ tải toàn bộ dữ liệu cần thiết một lần duy nhất
- Đưa ra đường dẫn hoặc thư mục cụ thể của Cache giúp tăng tốc độ xử lý
- Sử dụng multi-level cache cho nested dependency
- Đặt cache expiration dựa trên thay đổi file hoặc workflow tags

VD:
```yaml
jobs:
    build:
        runs-on: ubuntu-latest
        steps:
            - uses: actions/checkout@v2

            - name: Cache Node.js modules
              uses: actions/cache@v2
              with:
                path: node_modules
                key: ${{ runner.os }}-node-${{ hashFiles('**/package-lock.json') }}
                restore-keys: |
                  ${{ runner.os }}-node-
```

# Xác định các bước để đưa data giữa các jobs trong workflow

- Chia sẻ thông tin và artifacts giữa các jobs cải thiện tốc độ xử lý và giảm thời gian chạy workflow
- Định nghĩa output từ các bước riêng lẻ và pass dữ liệu
- Tận dụng các giá trị của environment cho secrets và configuration
- Quản lý tài nguyên sử dụng với job concurrency limits

VD:
```yaml
jobs:
    build:
        runs-on: ubuntu-lastest
        steps:
            - ... build steps ...
            - name: Upload build artifacts
              uses: actions/upload-artifact@v2
              with:
                name: application
                path: build/app.zip
        
    test:
        runs-on: ubuntu-lastest
        needs: build
        steps:
            - name: Download build artifacts
              uses: actions/download-artifact@v2
              with:
                name: application
            - name: Run tests
              ... test steps using the downloaded application ...
``` 

# Loại bỏ workflow artifacts không cần thiết từ GitHub

- Tự động loại bỏ các artifacts outdated hoặc không cần thiết
    - 90/30 ngày sau khi tạo public/private repo
- Tích hợp các cleanup actions với workflow triggers
- Sử dụng các chính sách lưu trữ (retention policies) khác nhau cho artifacts
- Tận dụng Github's built-in artifact management để quản lý chúng

VD:
```yaml
jobs:
    cleanup:
        runs-on: ubuntu-latest
        steps:
            - uses: actions/stale@v1
              with:
                days: 7
                repository: ${{ github.repository }}
                token: ${{ secrets.GITHUB_TOKEN }}
```

# Thêm workflow status badges

- Hiển thị trạng thái của workflow trên README hoặc trang web
- Tùy chỉnh badge để hiển thị trạng thái của workflow với màu hoặc icon
- Tích hợp badge vào dashboard hoặc trang docs của dự án
- Dynamic badge cho các trạng thái khác nhau của workflow, phù hợp với real-time status

VD:
``` bash
[![Build Status](https://img.shields.io/github/workflow/status/your-username/your-repo/build/master)](https://github.com/your-username/your-repo/actions)
```

# Thêm lớp bảo vệ cho environment

- Bảo vệ và kiểm soát quyền truy cập vào dữ liệu và tài nguyên
- Thực hiện đặc quyền tối thiểu cho các biến secret và environment
- Sử dụng masking để xáo trộn dữ liệu trong logs
- Tích hợp các công cụ bảo mật bên thứ 3 để kiểm tra và giám sát

VD:
```yaml
env:
    API_KEY: ${{ secrets.API_KEY }}

jobs:
    update-api-key:
        runs-on: ubuntu-latest
        steps:
            - name: Change API Key
              env:
                API_KEY: ${{ secrets.NEW_API_KEY }}
              requires:
                - approval
```

# Định nghĩa ma trận cho các jobs khác nhau

- Chạy nhiều phiên bản của cùng một job với các biến môi trường khác nhau trong cùng một workflow
- Sử dụng các bước tham số cho adaptable workflows
- Tận dụng kết hợp của ma trận cho efficient và scalable workflows
- Tích hợp ma trận với các tính năng khác để cải thiện tự động hóa

VD:
```yaml
jobs:
    test:
        strategy:
            matrix: # Create parallel jobs for different OS and Node.js versions
                os: [ubuntu-latest, windows-latest, macos-latest]
                node_version: [10, 12, 14]
        runs-on: ${{ matrix.os }}
        steps:
            - uses: actions/checkouts@v4
            - uses: actions/setup-node@v3
              with:
                node-version: ${{ matrix.node_version }}
            - name: Install dependencies
              run: npm ci
            - name: Run tests
              run: npm test
```

# Tích hợp workflow approval gates

- Yêu cầu phê duyệt thủ công trước các giai đoạn quan trọng của quy trình làm việc
- Sử dụng các bước approval để kiểm tra và xác nhận các thay đổi
- Tích hợp với Teams, Slack hoặc các công cụ khác để thông báo và xác nhận